In [2]:
import pandas as pd 
import numpy as np  
from datetime import datetime 
import datetime               
from datetime import timedelta 

In [3]:
class hwk_AI:
        
    def asf(df):
        # 컬럼명 영문화
        df.columns = ['w/c','shopdate','shift','code','down_class',
                      'fault','model','down_time','down_minute','cause',
                      'res_dept','management','Man_hour','number_of_vehicle']
        df['shift'] = df['shift'].replace(1,'주간')
        df['shift'] = df['shift'].replace(2,'야간')

        # 차종이름변경 및 불필요코드 삭제 (BN→JB, XX/TD→삭제, IH→UB, H8→YB)
        df['model'] = df['model'].replace('BN','JB')
        df['model'] = df['model'].replace('IH','UB')
        df['model'] = df['model'].replace('H8','YB')
        df = df[df['model'] != 'TD']
        df = df[df['model'] != 'XX']

        df['cause'] = df.cause.str[0:-9]

        df['start_time'] = df['down_time'].str.split('-').str[0] + ' '
        df['end_time'] = df['down_time'].str.split('-').str[1]

        df = df.reset_index(drop=True)

        df['shopdate'] = pd.to_datetime(df['shopdate'],format='%Y/%m/%d')       # 날짜를 문자에서 시간속성으로 변경

        # start_time이나 end_time에 '24'가 들어갔을 때, shopdate날짜에 1일을 더해주기
        for i in range(len(df)):                                                 
            if df.loc[i,'start_time'][:2] == '24' or df.loc[i,'end_time'][:2] == '24':
                df.loc[i,'shopdate'] = df.loc[i,'shopdate'] + datetime.timedelta(days=1)
        
        # 시간속성으로 변경한 날짜를 다시 문자속성으로 변경 
        # 왜냐하면 shopdate와 start_time 또는 end_time를 합치려면 같은 문자속성이어야 함
        for i in range(len(df)):
            df.loc[i,'shopdate'] = df.loc[i,'shopdate'].strftime('%Y/%m/%d')
    
        df['start_time'] = df['start_time'].str.replace('24','00')   # 중단시작시각의 '24'를 '00'으로 모두변경
        df['end_time'] = df['end_time'].str.replace('24','00')       # 중단종료시각의 '24'를 '00'으로 모두변경

        # start_time과 end_time의 날짜 붙여주기
        df['start_time'] = df['shopdate'] + ' ' + df['start_time']
        df['end_time'] = df['shopdate'] + ' ' + df['end_time']

        # 책임구 (r-dept) 영문코드삭제
        df['res_dept2'] = df['res_dept'].str[5:]
        
        return df